In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy.special import logit
pd.options.display.max_columns = 100

In [63]:
df = pd.read_csv('pitchers17.csv')
df

,pitcher,y,bf,bb,k,hr,h,lo,po,fo,go
0,Aardsma_David_430911,2012,5,1,1,1,1,0,0,0,2
1,Aardsma_David_430911,2013,172,13,36,7,39,12,8,28,28
2,Aardsma_David_430911,2015,126,11,35,6,25,9,5,21,17
3,Abad_Fernando_472551,2012,207,18,38,6,56,4,16,28,38
4,Abad_Fernando_472551,2013,166,10,32,3,42,11,17,22,29
...,...,...,...,...,...,...,...,...,...,...,...
4260,Zych_Tony_543964,2017,170,18,35,2,30,9,9,16,47
4261,deGrom_Jacob_594798,2014,563,41,144,7,117,32,19,67,126
4262,deGrom_Jacob_594798,2015,749,36,205,16,149,47,35,90,161
4263,deGrom_Jacob_594798,2016,604,36,143,15,142,36,25,68,137


In [64]:
bb_prob = sum(df.bb)/sum(df.bf)
so_prob = sum(df.k)/sum(df.bf)
krate_prob = sum(df.k)/(sum(df.bf) - sum(df.bb))
hr_prob = sum(df.hr)/sum(df.bf)
hrrate_prob = sum(df.hr)/(sum(df.bf)-sum(df.bb)-sum(df.k))
hrate_prob = (sum(df.h)-sum(df.hr))/(sum(df.bf)-sum(df.bb)-sum(df.k)-sum(df.hr))

bb_prob, so_prob, krate_prob, hr_prob, hrrate_prob, hrate_prob

(0.07491447652363538,
 0.2063503168798128,
 0.22306079994028244,
 0.027626467982645436,
 0.03843761614721209,
 0.2914712660905554)

In [65]:
df['krate'] = df.k/(df.bf-df.bb)

In [66]:
df16k = df.loc[(df.y == 2016) & (df.bf >=500)]
df16k = df16k[['pitcher','krate','bf']]
df16k.sort_values(by='krate', ascending = False).head(10)

,pitcher,krate,bf
1190,Fernandez_Jose_605228,0.370968,731
3524,Scherzer_Max_453286,0.335697,900
3721,Strasburg_Stephen_544931,0.330325,597
2021,Kershaw_Clayton_477132,0.322702,543
3775,Syndergaard_Noah_592789,0.310984,742
3183,Ray_Robbie_592662,0.309220,772
3445,Salazar_Danny_517593,0.309021,581
3942,Velasquez_Vincent_592826,0.300395,550
3964,Verlander_Justin_434378,0.300236,902
120,Archer_Chris_502042,0.297573,850


In [67]:
df17k = df.loc[(df.y == 2017) & (df.bf >=500)]
df17k = df17k[['pitcher','krate','bf']]
df17k.sort_values(by='krate', ascending = False).head(10)

,pitcher,krate,bf
3452,Sale_Chris_519242,0.381188,851
3525,Scherzer_Max_453286,0.369655,778
3184,Ray_Robbie_592662,0.367003,662
2058,Kluber_Corey_446372,0.357625,775
1754,Hill_Rich_448179,0.330020,551
2948,Peacock_Brad_502748,0.329243,546
121,Archer_Chris_502042,0.314394,852
3556,Severino_Luis_622663,0.314208,783
3722,Strasburg_Stephen_544931,0.311927,696
2022,Kershaw_Clayton_477132,0.311248,679


In [68]:
df['hrate'] = (df.h-df.hr)/(df.bf-df.bb-df.k-df.hr)

In [69]:
df16h = df.loc[(df.y == 2016) & (df.bf >=500)]
df16h = df16h[['pitcher','hrate','bf']]
df16h.sort_values(by='hrate', ascending = False).head(10)

,pitcher,hrate,bf
3183,Ray_Robbie_592662,0.347732,772
2939,Paxton_James_572020,0.346260,508
2958,Pelfrey_Mike_460059,0.341981,541
759,Cole_Gerrit_543037,0.339726,503
2980,Perdomo_Luis_606131,0.336066,655
2511,McHugh_Collin_543521,0.335185,795
412,Bradley_Archie_605151,0.334951,630
1047,Duffey_Tyler_608648,0.334118,593
4009,Wacha_Michael_608379,0.333333,600
3775,Syndergaard_Noah_592789,0.332627,742


In [70]:
df17h = df.loc[(df.y == 2017) & (df.bf >=500)]
df17h = df17h[['pitcher','hrate','bf']]
df17h.sort_values(by='hrate', ascending = False).head(10)

,pitcher,hrate,bf
2619,Montero_Rafael_606160,0.361345,545
3778,Taillon_Jameson_592791,0.348148,584
3223,Richard_Clayton_453385,0.346154,852
2722,Nelson_Jimmy_519076,0.333333,727
248,Bauer_Trevor_545333,0.333333,749
1361,Gausman_Kevin_592332,0.331471,816
2564,Miley_Wade_489119,0.329060,727
3055,Pivetta_Nick_601713,0.328729,584
788,Colon_Bartolo_112526,0.328629,648
396,Boyd_Matt_571510,0.327830,602


In [321]:
df['bbrate'] = (df.bb)/(df.bf)
df['hrrate'] = (df.hr)/(df.bf-df.bb-df.k)
df['lorate'] = (df.lo)/(df.lo+df.po+df.fo+df.go)
df['porate'] = (df.po)/(df.lo+df.po+df.fo+df.go)
df['forate'] = (df.fo)/(df.lo+df.po+df.fo+df.go)
df['gorate'] = (df.go)/(df.lo+df.po+df.fo+df.go)
df['eorate'] = (df.go + df.po)/(df.lo+df.po+df.fo+df.go)
df['pdrate'] = (df.lo + df.fo)/(df.lo+df.po+df.fo+df.go)

df['kprop'] = (df.k)/(df.lo+df.po+df.fo+df.go+df.k)
df['loprop'] = (df.lo)/(df.lo+df.po+df.fo+df.go+df.k)
df['foprop'] = (df.fo)/(df.lo+df.po+df.fo+df.go+df.k)
df['goprop'] = (df.go)/(df.lo+df.po+df.fo+df.go+df.k)
df['poprop'] = (df.po)/(df.lo+df.po+df.fo+df.go+df.k)
df['eoprop'] = (df.go+df.po)/(df.lo+df.po+df.fo+df.go+df.k)
df['pdprop'] = (df.lo+df.fo)/(df.lo+df.po+df.fo+df.go+df.k)

In [113]:
df

,pitcher,y,bf,bb,k,hr,h,lo,po,fo,go,krate,hrate,bbrate,hrrate
0,Aardsma_David_430911,2012,5,1,1,1,1,0,0,0,2,0.250000,0.000000,0.200000,0.333333
1,Aardsma_David_430911,2013,172,13,36,7,39,12,8,28,28,0.226415,0.275862,0.075581,0.056911
2,Aardsma_David_430911,2015,126,11,35,6,25,9,5,21,17,0.304348,0.256757,0.087302,0.075000
3,Abad_Fernando_472551,2012,207,18,38,6,56,4,16,28,38,0.201058,0.344828,0.086957,0.039735
4,Abad_Fernando_472551,2013,166,10,32,3,42,11,17,22,29,0.205128,0.322314,0.060241,0.024194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4260,Zych_Tony_543964,2017,170,18,35,2,30,9,9,16,47,0.230263,0.243478,0.105882,0.017094
4261,deGrom_Jacob_594798,2014,563,41,144,7,117,32,19,67,126,0.275862,0.296496,0.072824,0.018519
4262,deGrom_Jacob_594798,2015,749,36,205,16,149,47,35,90,161,0.287518,0.270325,0.048064,0.031496
4263,deGrom_Jacob_594798,2016,604,36,143,15,142,36,25,68,137,0.251761,0.309756,0.059603,0.035294


In [82]:
from itertools import cycle

years = cycle([2012,2013,2014,2015,2016,2017])

df2 = pd.DataFrame(df.pitcher.unique())
df2 = pd.concat([df2]*6, ignore_index=True)
df2.columns = ['pitcher']
df2 = df2.sort_values(by='pitcher')
df2['y'] = [next(years) for i in range(len(df2))]
df2


,pitcher,y
0,Aardsma_David_430911,2012
6108,Aardsma_David_430911,2013
4581,Aardsma_David_430911,2014
3054,Aardsma_David_430911,2015
1527,Aardsma_David_430911,2016
...,...,...
6107,deGrom_Jacob_594798,2013
3053,deGrom_Jacob_594798,2014
1526,deGrom_Jacob_594798,2015
7634,deGrom_Jacob_594798,2016


In [322]:
df3 = pd.merge(df2, df, on = ['pitcher','y'], how = 'left')
df3 = df3.fillna(0)
df3['bbrate_prev'] = df3.groupby('pitcher').bbrate.transform(lambda x : x.shift(1, fill_value = 0))
df3['krate_prev'] = df3.groupby('pitcher').krate.transform(lambda x : x.shift(1, fill_value = 0))
df3['hrate_prev'] = df3.groupby('pitcher').hrate.transform(lambda x : x.shift(1, fill_value = 0))
df3['hrrate_prev'] = df3.groupby('pitcher').hrrate.transform(lambda x : x.shift(1, fill_value = 0))
df3['bf_prev'] = df3.groupby('pitcher').bf.transform(lambda x : x.shift(1, fill_value = 0))
df3['lorate_prev'] = df3.groupby('pitcher').lorate.transform(lambda x : x.shift(1, fill_value = 0))
df3['porate_prev'] = df3.groupby('pitcher').porate.transform(lambda x : x.shift(1, fill_value = 0))
df3['forate_prev'] = df3.groupby('pitcher').forate.transform(lambda x : x.shift(1, fill_value = 0))
df3['gorate_prev'] = df3.groupby('pitcher').gorate.transform(lambda x : x.shift(1, fill_value = 0))
df3['eorate_prev'] = df3.groupby('pitcher').eorate.transform(lambda x : x.shift(1, fill_value = 0))
df3['pdrate_prev'] = df3.groupby('pitcher').pdrate.transform(lambda x : x.shift(1, fill_value = 0))

df3['kprop_prev'] = df3.groupby('pitcher').kprop.transform(lambda x : x.shift(1, fill_value = 0))
df3['loprop_prev'] = df3.groupby('pitcher').loprop.transform(lambda x : x.shift(1, fill_value = 0))
df3['poprop_prev'] = df3.groupby('pitcher').poprop.transform(lambda x : x.shift(1, fill_value = 0))
df3['foprop_prev'] = df3.groupby('pitcher').foprop.transform(lambda x : x.shift(1, fill_value = 0))
df3['goprop_prev'] = df3.groupby('pitcher').goprop.transform(lambda x : x.shift(1, fill_value = 0))
df3['eoprop_prev'] = df3.groupby('pitcher').eoprop.transform(lambda x : x.shift(1, fill_value = 0))
df3['pdprop_prev'] = df3.groupby('pitcher').pdprop.transform(lambda x : x.shift(1, fill_value = 0))


df3['bb_prev'] = df3.groupby('pitcher').bb.transform(lambda x : x.shift(1, fill_value = 0))
df3['k_prev'] = df3.groupby('pitcher').k.transform(lambda x : x.shift(1, fill_value = 0))
df3['hr_prev'] = df3.groupby('pitcher').hr.transform(lambda x : x.shift(1, fill_value = 0))
df3['lo_prev'] = df3.groupby('pitcher').lo.transform(lambda x : x.shift(1, fill_value = 0))
df3['po_prev'] = df3.groupby('pitcher').po.transform(lambda x : x.shift(1, fill_value = 0))
df3['fo_prev'] = df3.groupby('pitcher').fo.transform(lambda x : x.shift(1, fill_value = 0))
df3['go_prev'] = df3.groupby('pitcher').go.transform(lambda x : x.shift(1, fill_value = 0))
df3['h_prev'] = df3.groupby('pitcher').h.transform(lambda x : x.shift(1, fill_value = 0))

df3 = df3.loc[(df3.y != 2012) & (df3.bf >=200) & (df3.bf_prev >= 200)]

In [305]:
df3

,pitcher,y,bf,bb,k,hr,h,lo,po,fo,go,krate,hrate,bbrate,hrrate,lorate,porate,forate,gorate,eorate,kprop,loprop,foprop,goprop,poprop,eoprop,pdprop,pdrate,bbrate_prev,krate_prev,hrate_prev,hrrate_prev,bf_prev,lorate_prev,porate_prev,forate_prev,gorate_prev,eorate_prev,pdrate_prev,kprop_prev,loprop_prev,poprop_prev,foprop_prev,goprop_prev,eoprop_prev,pdprop_prev,bb_prev,k_prev,hr_prev,lo_prev,po_prev,fo_prev,go_prev,h_prev
9,Abad_Fernando_472551,2015,202.0,16.0,45.0,11.0,45.0,19.0,10.0,27.0,33.0,0.241935,0.261538,0.079208,0.078014,0.213483,0.112360,0.303371,0.370787,0.483146,0.335821,0.141791,0.201493,0.246269,0.074627,0.320896,0.343284,0.516854,0.056338,0.253731,0.205479,0.026667,213.0,0.149533,0.140187,0.299065,0.411215,0.551402,0.448598,0.322785,0.101266,0.094937,0.202532,0.278481,0.373418,0.303797,12.0,51.0,4.0,16.0,15.0,32.0,44.0,34.0
65,Adleman_Timothy_534947,2017,530.0,50.0,108.0,29.0,124.0,29.0,33.0,78.0,95.0,0.225000,0.276968,0.094340,0.077957,0.123404,0.140426,0.331915,0.404255,0.544681,0.314869,0.084548,0.227405,0.276968,0.096210,0.373178,0.311953,0.455319,0.066434,0.176030,0.246377,0.059091,286.0,0.160839,0.097902,0.356643,0.384615,0.482517,0.517483,0.247368,0.121053,0.073684,0.268421,0.289474,0.363158,0.389474,19.0,47.0,13.0,23.0,14.0,51.0,55.0,64.0
85,Albers_Matt_458006,2013,259.0,20.0,35.0,2.0,57.0,16.0,3.0,20.0,99.0,0.146444,0.272277,0.077220,0.009804,0.115942,0.021739,0.144928,0.717391,0.739130,0.202312,0.092486,0.115607,0.572254,0.017341,0.589595,0.208092,0.260870,0.079832,0.200913,0.222892,0.051429,238.0,0.076271,0.067797,0.220339,0.635593,0.703390,0.296610,0.271605,0.055556,0.049383,0.160494,0.462963,0.512346,0.216049,19.0,44.0,9.0,9.0,8.0,26.0,75.0,46.0
89,Albers_Matt_458006,2017,233.0,17.0,63.0,6.0,35.0,16.0,12.0,19.0,61.0,0.291667,0.197279,0.072961,0.039216,0.148148,0.111111,0.175926,0.564815,0.675926,0.368421,0.093567,0.111111,0.356725,0.070175,0.426901,0.204678,0.324074,0.076271,0.137615,0.320225,0.053191,236.0,0.121495,0.102804,0.252336,0.523364,0.626168,0.373832,0.218978,0.094891,0.080292,0.197080,0.408759,0.489051,0.291971,18.0,30.0,10.0,13.0,11.0,27.0,56.0,67.0
92,Alburquerque_Al_456379,2014,235.0,20.0,63.0,7.0,46.0,10.0,12.0,30.0,49.0,0.293023,0.268966,0.085106,0.046053,0.099010,0.118812,0.297030,0.485149,0.603960,0.384146,0.060976,0.182927,0.298780,0.073171,0.371951,0.243902,0.396040,0.134884,0.376344,0.306306,0.043103,215.0,0.082192,0.136986,0.301370,0.479452,0.616438,0.383562,0.489510,0.041958,0.069930,0.153846,0.244755,0.314685,0.195804,29.0,70.0,5.0,6.0,10.0,22.0,35.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,Zimmermann_Jordan_519455,2017,711.0,42.0,103.0,29.0,204.0,54.0,49.0,111.0,135.0,0.153961,0.325885,0.059072,0.051237,0.154728,0.140401,0.318052,0.386819,0.527221,0.227876,0.119469,0.245575,0.298673,0.108407,0.407080,0.365044,0.472779,0.057778,0.155660,0.302326,0.039106,450.0,0.075221,0.159292,0.292035,0.473451,0.632743,0.367257,0.226027,0.058219,0.123288,0.226027,0.366438,0.489726,0.284247,26.0,66.0,14.0,17.0,36.0,66.0,107.0,118.0
9145,Zito_Barry_217096,2013,606.0,52.0,86.0,19.0,173.0,36.0,34.0,91.0,115.0,0.155235,0.342984,0.085809,0.040598,0.130435,0.123188,0.329710,0.416667,0.539855,0.237569,0.099448,0.251381,0.317680,0.093923,0.411602,0.350829,0.460145,0.080706,0.156379,0.278992,0.032520,793.0,0.065657,0.146465,0.358586,0.429293,0.575758,0.424242,0.223529,0.050980,0.113725,0.278431,0.333333,0.447059,0.329412,64.0,114.0,20.0,26.0,58.0,142.0,170.0,186.0
9159,deGrom_Jacob_594798,2015,749.0,36.0,205.0,16.0,149.0,47.0,35.0,90.0,161.0,0.287518,0.270325,0.048064,0.031496,0.141141,0.105105,0.270270,0.483483,0.588589,0.381041,0.087361,0.167286,0.299257,0.065056,0.364312,0.254647,0.411411,0.072824,0.275862,0.296496,0.018519,563.0,0.131148,0.077869,0.274590,0.516393,0.594262,0.405738,0.371134,0.082474,0.048969,0.172

In [323]:
## krate 
smf.ols('bbrate ~ bbrate_prev + krate_prev + forate_prev',df3).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 bbrate   R-squared:                       0.296
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     166.0
Date:                Thu, 01 Apr 2021   Prob (F-statistic):           7.95e-90
Time:                        12:33:27   Log-Likelihood:                 3043.1
No. Observations:                1189   AIC:                            -6078.
Df Residuals:                    1185   BIC:                            -6058.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0327      0.003     10.156      0.000       0.026       0.039
bbrate_prev     0.5300      0.025     20.812      0.000       0.480       0.580
krate_prev      0.0324      0.009      3.670      0.000       0.015       0.050
forate_prev    -0.0204      0.009     -2.384      0.017      -0.037      -0.004
==============================================================================
Omnibus:                       26.284   Durbin-Watson:                   2.055
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.111
Skew:                           0.304   Prob(JB):                     2.89e-07
Kurtosis:                       3.488   Cond. No.                         50.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [336]:
smf.ols('krate ~ krate_prev +bbrate_prev + lorate_prev',df3).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  krate   R-squared:                       0.560
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     503.6
Date:                Thu, 01 Apr 2021   Prob (F-statistic):          6.33e-211
Time:                        12:42:32   Log-Likelihood:                 2075.8
No. Observations:                1189   AIC:                            -4144.
Df Residuals:                    1185   BIC:                            -4123.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0453      0.007      6.595      0.000       0.032       0.059
krate_prev      0.7393      0.020     37.531      0.000       0.701       0.778
bbrate_prev     0.0804      0.057      1.399      0.162      -0.032       0.193
lorate_prev     0.0798      0.037      2.174      0.030       0.008       0.152
==============================================================================
Omnibus:                       83.089   Durbin-Watson:                   2.154
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              116.132
Skew:                           0.578   Prob(JB):                     6.06e-26
Kurtosis:                       4.003   Cond. No.                         48.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [337]:
##lorate, eorate
smf.ols('hrrate ~ hrrate_prev + lorate_prev + forate_prev',df3).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 hrrate   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     62.92
Date:                Thu, 01 Apr 2021   Prob (F-statistic):           9.45e-38
Time:                        12:44:38   Log-Likelihood:                 3413.1
No. Observations:                1189   AIC:                            -6818.
Df Residuals:                    1185   BIC:                            -6798.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0064      0.002      2.642      0.008       0.002       0.011
hrrate_prev     0.2046      0.032      6.326      0.000       0.141       0.268
lorate_prev     0.0988      0.012      8.098      0.000       0.075       0.123
forate_prev     0.0463      0.007      6.919      0.000       0.033       0.059
==============================================================================
Omnibus:                       31.283   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.550
Skew:                           0.360   Prob(JB):                     3.14e-08
Kurtosis:                       3.423   Cond. No.                         85.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [326]:
##porate, krate
smf.ols('hrate ~ hrate_prev + porate_prev + krate_prev',df3).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  hrate   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     28.74
Date:                Thu, 01 Apr 2021   Prob (F-statistic):           6.14e-18
Time:                        12:33:34   Log-Likelihood:                 2473.6
No. Observations:                1189   AIC:                            -4939.
Df Residuals:                    1185   BIC:                            -4919.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.2772      0.010     28.579      0.000       0.258       0.296
hrate_prev      0.1294      0.029      4.471      0.000       0.073       0.186
porate_prev    -0.0981      0.022     -4.388      0.000      -0.142      -0.054
krate_prev     -0.0620      0.014     -4.333      0.000      -0.090      -0.034
==============================================================================
Omnibus:                       12.947   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               19.932
Skew:                           0.040   Prob(JB):                     4.70e-05
Kurtosis:                       3.629   Cond. No.                         37.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""